In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.fft import fft, ifft
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks

In [2]:
def ewt_boundaries_equal_energy(spectrum, N):
    energy = np.cumsum(spectrum) / np.sum(spectrum)
    boundaries = []
    for k in range(1, N):
        idx = np.argmin(np.abs(energy - k / N))
        boundaries.append(idx / len(spectrum) * np.pi)
    return boundaries

def ewt_boundaries(spectrum, N, smooth_sigma=2):
    """Spectrum-peak-based boundaries (paper-like)."""
    spectrum_smooth = gaussian_filter1d(spectrum, sigma=smooth_sigma)
    if np.allclose(spectrum_smooth, spectrum_smooth[0]):
        return ewt_boundaries_equal_energy(spectrum, N)
    peaks, _ = find_peaks(spectrum_smooth)
    if len(peaks) < N - 1:
        return ewt_boundaries_equal_energy(spectrum, N)
    amps = spectrum_smooth[peaks]
    top_peaks = sorted([p for _, p in sorted(zip(amps, peaks), reverse=True)][:N-1])
    boundaries = [p / len(spectrum) * np.pi for p in top_peaks]
    return boundaries

def make_filter_bank(boundaries, L):
    freqs = np.linspace(0, np.pi, L//2 + 1)
    mfb = []
    # scaling (lowpass)
    phi = np.zeros_like(freqs)
    phi[freqs <= boundaries[0]] = 1
    mfb.append(phi)
    # wavelet bands
    for i in range(len(boundaries)):
        psi = np.zeros_like(freqs)
        if i == len(boundaries) - 1:
            mask = (freqs > boundaries[i])
        else:
            mask = (freqs > boundaries[i]) & (freqs <= boundaries[i+1])
        psi[mask] = 1
        mfb.append(psi)
    return mfb

def EWT1D(signal, N=3, smooth_sigma=2):
    """
    Empirical Wavelet Transform (simple SciPy-based).
    Returns:
      modes: np.array shape (N, L)
      mfb: list of half-spectrum filters (for IEWT usage)
      boundaries: list of radian boundaries
    """
    L = len(signal)
    spectrum_half = np.abs(fft(signal))[:L//2 + 1]
    boundaries = ewt_boundaries(spectrum_half, N, smooth_sigma=smooth_sigma)
    mfb = make_filter_bank(boundaries, L)
    modes = []
    S_full = fft(signal)
    for filt in mfb:
        filt_full = np.concatenate([filt, filt[-2:0:-1]])
        mode_freq = S_full * filt_full
        mode_time = np.real(ifft(mode_freq))
        modes.append(mode_time)
    return np.array(modes), mfb, boundaries

def iEWT1D(modes, mfb):
    """
    Inverse EWT (simple reconstruction by summation of modes).
    modes: (n_modes, L_test) or (n_modes, L)
    mfb is unused here but kept for API similarity.
    """
    return np.sum(modes, axis=0)

In [3]:
class RELM:
    def __init__(self, n_hidden=100, activation='tanh', C=1.0, random_state=None):
        self.n_hidden = int(n_hidden)
        self.activation = activation
        self.C = float(C)
        self.random_state = random_state
        self.is_fitted = False

    def _init_weights(self, n_features):
        rng = np.random.default_rng(self.random_state)
        self.W = rng.uniform(-1, 1, size=(self.n_hidden, n_features))
        self.b = rng.uniform(-1, 1, size=(self.n_hidden,))

    def _activation(self, X):
        if self.activation == 'sigmoid':
            X = np.clip(X, -500, 500)
            return 1.0 / (1.0 + np.exp(-X))
        if self.activation == 'tanh':
            return np.tanh(X)
        if self.activation == 'relu':
            return np.maximum(0.0, X)
        raise ValueError("Unknown activation")

    def fit(self, X, y):
        X = np.asarray(X); y = np.asarray(y)
        if y.ndim == 1:
            y = y.reshape(-1,1)
        N, d = X.shape
        self._init_weights(d)
        H = self._activation(X @ self.W.T + self.b)
        if N >= self.n_hidden:
            A = (np.eye(self.n_hidden) / self.C) + (H.T @ H)
            B = H.T @ y
            self.beta = np.linalg.solve(A, B)
        else:
            A = (np.eye(N) / self.C) + (H @ H.T)
            B = y
            self.beta = H.T @ np.linalg.solve(A, B)
        self.is_fitted = True
        return self

    def predict(self, X):
        if not self.is_fitted:
            raise RuntimeError("Model not fitted.")
        H = self._activation(np.asarray(X) @ self.W.T + self.b)
        Y = H @ self.beta
        return Y.ravel() if Y.shape[1] == 1 else Y


In [4]:
def create_multivariate_lagged_dataset(df, target_col, feature_cols, lag=3):
    data = df[feature_cols].values
    target_idx = feature_cols.index(target_col)
    X, y = [], []
    for i in range(lag, len(df)):
        X.append(data[i-lag:i].flatten())
        y.append(data[i, target_idx])
    return np.array(X), np.array(y)

def safe_mape(y_true, y_pred, min_denom=1.0):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    mask = np.abs(y_true) >= min_denom
    if np.sum(mask) == 0:
        return np.nan
    return float(np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100)

def sde(y_true, y_pred):
    return float(np.std(np.asarray(y_true) - np.asarray(y_pred)))

In [5]:
class GWO:
    def __init__(self, obj_func, lb, ub, dim, n_agents=12, n_iter=25, seed=42):
        self.obj_func = obj_func
        self.lb = np.array(lb, dtype=float)
        self.ub = np.array(ub, dtype=float)
        self.dim = dim
        self.n_agents = n_agents
        self.n_iter = n_iter
        self.rng = np.random.default_rng(seed)

    def optimize(self):
        wolves = self.rng.uniform(self.lb, self.ub, size=(self.n_agents, self.dim))
        fitness = np.array([self.obj_func(w) for w in wolves])
        idx = np.argsort(fitness)
        alpha, beta, delta = wolves[idx[0]].copy(), wolves[idx[1]].copy(), wolves[idx[2]].copy()
        f_alpha, f_beta, f_delta = float(fitness[idx[0]]), float(fitness[idx[1]]), float(fitness[idx[2]])
        for t in range(self.n_iter):
            a = 2 - 2 * (t / (self.n_iter - 1 + 1e-9))
            for i in range(self.n_agents):
                X = wolves[i].copy()
                for j in range(self.dim):
                    r1, r2 = self.rng.random(), self.rng.random()
                    A1 = 2 * a * r1 - a; C1 = 2 * r2
                    D_alpha = abs(C1 * alpha[j] - X[j]); X1 = alpha[j] - A1 * D_alpha
                    r1, r2 = self.rng.random(), self.rng.random()
                    A2 = 2 * a * r1 - a; C2 = 2 * r2
                    D_beta = abs(C2 * beta[j] - X[j]); X2 = beta[j] - A2 * D_beta
                    r1, r2 = self.rng.random(), self.rng.random()
                    A3 = 2 * a * r1 - a; C3 = 2 * r2
                    D_delta = abs(C3 * delta[j] - X[j]); X3 = delta[j] - A3 * D_delta
                    X[j] = (X1 + X2 + X3) / 3.0
                wolves[i] = np.clip(X, self.lb, self.ub)
            fitness = np.array([self.obj_func(w) for w in wolves])
            idx = np.argsort(fitness)
            if fitness[idx[0]] < f_alpha:
                alpha, f_alpha = wolves[idx[0]].copy(), float(fitness[idx[0]])
            if fitness[idx[1]] < f_beta:
                beta, f_beta = wolves[idx[1]].copy(), float(fitness[idx[1]])
            if fitness[idx[2]] < f_delta:
                delta, f_delta = wolves[idx[2]].copy(), float(fitness[idx[2]])
        return alpha, f_alpha

In [6]:
def decode_relm_params(position, Hmin=20, Hmax=500, Cmin_log=-4, Cmax_log=4):
    # position expected: [h_raw (0..1), c_log (Cmin_log..Cmax_log), act_raw (0..2)]
    h_raw, c_log, a_raw = position
    n_hidden = int(np.round(Hmin + np.clip(h_raw, 0, 1) * (Hmax - Hmin)))
    n_hidden = int(np.clip(n_hidden, Hmin, Hmax))
    C = 10.0 ** float(np.clip(c_log, Cmin_log, Cmax_log))
    act_idx = int(np.round(np.clip(a_raw, 0, 2)))
    activation = ['tanh', 'sigmoid', 'relu'][act_idx]
    return n_hidden, C, activation

def make_relm_objective(X_train, y_train, X_val, y_val, random_state=42):
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_val_s = scaler.transform(X_val)
    def objective(position):
        n_hidden, C, activation = decode_relm_params(position)
        try:
            model = RELM(n_hidden=n_hidden, activation=activation, C=C, random_state=random_state)
            model.fit(X_train_s, y_train)
            y_pred = model.predict(X_val_s)
            return float(np.sqrt(mean_squared_error(y_val, y_pred)))
        except Exception:
            return 1e6
    return objective

In [7]:
def ewt_gwo_relm_iewt_pipeline(
    df,
    target_column,
    feature_columns,
    lag_steps=12,
    n_modes=4,
    gwo_agents=12,
    gwo_iters=25,
    random_state=42,
    Hmin=20, Hmax=500,
    Cmin_log=-4, Cmax_log=4,
    smooth_sigma=2,
    max_step_eval=7
):
    """
    Returns:
      - per_mode_info: list of dicts with best params for each mode
      - one_step_metrics: dict of combined reconstructed one-step metrics
      - multistep_df: DataFrame with combined reconstructed multi-step metrics (Step 1..H)
      - boundaries (radian)
    """
    # EWT on full signal (target)
    signal = df[target_column].values
    modes, mfb, boundaries = EWT1D(signal, N=n_modes, smooth_sigma=smooth_sigma)

    # time-aware splits after constructing lagged X (same across modes)
    n_samples = len(signal) - lag_steps
    if n_samples <= 0:
        raise ValueError("lag_steps too large for series length.")
    train_end = int(0.7 * n_samples)
    val_end   = int(0.85 * n_samples)

    per_mode_info = []
    test_mode_preds = []     # will collect per-mode predicted test sequences
    test_mode_lengths = []   # to keep track of test lengths per mode

    # For each mode: tune, train final on train+val, predict test mode
    for mode_idx in range(n_modes):
        mode_series = modes[mode_idx]
        df_mode = df.copy()
        df_mode[target_column] = mode_series

        X_mode, y_mode = create_multivariate_lagged_dataset(df_mode, target_column, feature_columns, lag=lag_steps)
        X_train, y_train = X_mode[:train_end], y_mode[:train_end]
        X_val, y_val     = X_mode[train_end:val_end], y_mode[train_end:val_end]
        X_test, y_test   = X_mode[val_end:], y_mode[val_end:]

        # Basic checks
        if len(X_train) < 5 or len(X_val) < 1 or len(X_test) < 1:
            print(f"[Mode {mode_idx+1}] insufficient data for splits; skipping this mode.")
            per_mode_info.append({
                "mode": mode_idx+1, "skipped": True
            })
            # append zeros/padding so IEWT reconstruct shape consistent: use zeros for this mode's test predictions
            test_mode_preds.append(np.zeros_like(X_test[:,0]) if X_test.size else np.array([]))
            test_mode_lengths.append(len(X_test))
            continue

        # Build GWO objective (trained on train -> validated on val)
        obj = make_relm_objective(X_train, y_train, X_val, y_val, random_state=random_state)

        lb = np.array([0.0, Cmin_log, 0.0], dtype=float)
        ub = np.array([1.0, Cmax_log, 2.0], dtype=float)
        gwo = GWO(obj, lb, ub, dim=3, n_agents=gwo_agents, n_iter=gwo_iters, seed=random_state+mode_idx)

        best_pos, best_fit = gwo.optimize()
        n_hidden, C, activation = decode_relm_params(best_pos, Hmin=Hmin, Hmax=Hmax, Cmin_log=Cmin_log, Cmax_log=Cmax_log)

        # Train final model on train+val, evaluate test predictions for this mode
        scaler = StandardScaler()
        X_trval = np.vstack([X_train, X_val])
        y_trval = np.concatenate([y_train, y_val])
        X_trval_s = scaler.fit_transform(X_trval)
        X_test_s = scaler.transform(X_test)

        model_final = RELM(n_hidden=n_hidden, activation=activation, C=C, random_state=random_state)
        model_final.fit(X_trval_s, y_trval)
        y_pred_test_mode = model_final.predict(X_test_s)

        per_mode_info.append({
            "mode": mode_idx+1,
            "best_val_rmse": float(best_fit),
            "n_hidden": int(n_hidden),
            "C": float(C),
            "activation": activation,
            "test_len": int(len(y_test))
        })

        test_mode_preds.append(y_pred_test_mode)
        test_mode_lengths.append(len(y_test))

    # Ensure all mode test predictions align in length (they should if same splits used)
    # Convert test_mode_preds -> array shape (n_modes, n_test)
    # Some modes may have been skipped and contain zeros/empty arrays; handle gracefully.
    # Find maximum available test length among modes (should be same)
    n_test = None
    for ln in test_mode_lengths:
        if ln:
            n_test = ln
            break
    if n_test is None:
        raise RuntimeError("No valid test data across modes.")

    # Stack predictions; for any shorter arrays (shouldn't happen), pad with zeros at end
    stacked_preds = []
    for arr in test_mode_preds:
        if len(arr) == n_test:
            stacked_preds.append(arr)
        elif len(arr) == 0:
            stacked_preds.append(np.zeros(n_test))
        else:
            # pad or trim to n_test
            a = np.asarray(arr)
            if len(a) < n_test:
                padded = np.concatenate([a, np.zeros(n_test - len(a))])
                stacked_preds.append(padded)
            else:
                stacked_preds.append(a[:n_test])
    stacked_preds = np.array(stacked_preds)  # shape (n_modes, n_test)

    # IEWT reconstruction (sum of predicted modes)
    y_pred_reconstructed = iEWT1D(stacked_preds, mfb=None)
    # True test target values: original df target from lag_steps + val_end onwards
    y_true_test = df[target_column].values[lag_steps + val_end : lag_steps + val_end + n_test]

    # One-step metrics (combined reconstructed)
    mae = mean_absolute_error(y_true_test, y_pred_reconstructed)
    rmse = np.sqrt(mean_squared_error(y_true_test, y_pred_reconstructed))
    mape = safe_mape(y_true_test, y_pred_reconstructed)
    sdev = sde(y_true_test, y_pred_reconstructed)

    one_step_metrics = {"MAE": float(mae), "RMSE": float(rmse), "MAPE (%)": float(mape) if not np.isnan(mape) else np.nan, "SDE": float(sdev)}

    # Multi-step direct predictions: for each horizon, per-mode predict direct, then IEWT reconstruct step predictions
    multistep_rows = []
    for step in range(1, max_step_eval + 1):
        # For each mode, build X_test inputs that can predict t+step: X_mode[val_end : val_end + (n_test - step)]
        mode_step_preds = []
        valid = True
        for mode_idx in range(n_modes):
            info = per_mode_info[mode_idx]
            if info.get("skipped", False):
                # predict zeros for this mode
                mode_step_preds.append(np.zeros(max(0, n_test - step)))
                continue

            # Rebuild mode dataset
            mode_series = modes[mode_idx]
            df_mode = df.copy()
            df_mode[target_column] = mode_series
            X_mode, y_mode = create_multivariate_lagged_dataset(df_mode, target_column, feature_columns, lag=lag_steps)

            # Splits
            X_train, y_train = X_mode[:train_end], y_mode[:train_end]
            X_val, y_val     = X_mode[train_end:val_end], y_mode[train_end:val_end]
            X_test_all, y_test_all = X_mode[val_end:], y_mode[val_end:]

            # Determine X_test inputs for this step
            if X_test_all.shape[0] <= step:
                valid = False
                break
            X_test_step = X_test_all[:-step]          # inputs available
            y_test_step = y_test_all[step:]           # corresponding true targets (not used until final eval)

            # Train model on TRAIN ONLY using best params for this mode
            n_hidden = info.get("n_hidden", Hmin)
            C = info.get("C", 1.0)
            activation = info.get("activation", "tanh")

            scaler_train = StandardScaler()
            X_train_s = scaler_train.fit_transform(X_train)
            X_test_step_s = scaler_train.transform(X_test_step)

            model_step = RELM(n_hidden=n_hidden, activation=activation, C=C, random_state=random_state)
            model_step.fit(X_train_s, y_train)

            y_pred_step_mode = model_step.predict(X_test_step_s)  # length = n_test - step
            mode_step_preds.append(y_pred_step_mode)

        if not valid:
            # Not enough data for this step across modes; skip remaining steps
            break

        mode_step_preds = np.array(mode_step_preds)  # shape (n_modes, n_test-step)
        y_pred_step_reconstructed = iEWT1D(mode_step_preds, mfb=None)
        y_true_step = df[target_column].values[lag_steps + val_end + step : lag_steps + val_end + step + (y_pred_step_reconstructed.shape[0])]

        # Note: make sure lengths align; y_true_step length should equal y_pred_step_reconstructed length
        L_true = len(y_true_step)
        L_pred = len(y_pred_step_reconstructed)
        if L_true != L_pred:
            # trim to min length
            m = min(L_true, L_pred)
            y_true_step = y_true_step[:m]
            y_pred_step_reconstructed = y_pred_step_reconstructed[:m]

        multistep_rows.append({
            "Step": step,
            "MAE": float(mean_absolute_error(y_true_step, y_pred_step_reconstructed)),
            "RMSE": float(np.sqrt(mean_squared_error(y_true_step, y_pred_step_reconstructed))),
            "MAPE (%)": float(safe_mape(y_true_step, y_pred_step_reconstructed)),
            "SDE": float(sde(y_true_step, y_pred_step_reconstructed))
        })

    multistep_df = pd.DataFrame(multistep_rows)

    return {
        "per_mode_info": per_mode_info,
        "one_step_metrics": one_step_metrics,
        "multistep_df": multistep_df,
        "boundaries_rad": boundaries,
        "y_true_test": y_true_test,
        "y_pred_reconstructed": y_pred_reconstructed
    }


In [8]:
feature_columns = ['AirTemp','Azimuth','CloudOpacity','DewpointTemp','Dhi','Dni','Ebh',
                   'WindDirection10m','Ghi','RelativeHumidity','SurfacePressure','WindSpeed10m']
df = pd.read_csv('/Users/hrishityelchuri/Documents/windPred/raw/8.52 hrishit data.csv')

df['PeriodEnd'] = pd.to_datetime(df['PeriodEnd'])
df['PeriodStart'] = pd.to_datetime(df['PeriodStart'])
df = df.sort_values('PeriodEnd')

In [9]:
res = ewt_gwo_relm_iewt_pipeline(
    df,
    target_column='WindSpeed10m',
    feature_columns=feature_columns,
    lag_steps=12,
    n_modes=4,
    gwo_agents=12,
    gwo_iters=25,
    random_state=42,
    Hmin=20, Hmax=500,
    Cmin_log=-4, Cmax_log=4,
    smooth_sigma=2,
    max_step_eval=7
)
print("Per-mode best params:")
print(pd.DataFrame(res['per_mode_info']))
print("\nOne-step reconstructed metrics:")
print(res['one_step_metrics'])
print("\nMulti-step reconstructed metrics:")
print(res['multistep_df'].to_string(index=False))

Per-mode best params:
   mode  best_val_rmse  n_hidden           C activation  test_len
0     1       0.281201       500  834.073563       relu     20612
1     2       0.048985       500    5.053869       relu     20612
2     3       0.175962       491    0.027432       relu     20612
3     4       0.240902       494   28.884770       relu     20612

One-step reconstructed metrics:
{'MAE': 0.3259556682345028, 'RMSE': 0.4157471298598639, 'MAPE (%)': 13.234027214051864, 'SDE': 0.412707331878875}

Multi-step reconstructed metrics:
 Step      MAE     RMSE  MAPE (%)      SDE
    1 0.500203 0.653093 20.273487 0.631078
    2 0.642642 0.834021 25.342455 0.816892
    3 0.771557 0.994560 29.826908 0.980232
    4 0.883250 1.129299 33.791592 1.116699
    5 0.979764 1.242604 37.279853 1.231154
    6 1.062909 1.338970 40.340968 1.328337
    7 1.135306 1.418880 43.071595 1.408835
